# Logística de envíos: ¿Cuándo llega?

## Mentoría DiploDatos 2019 

### Integrantes:

- Alini, Walter
- Frau, Johanna
- Salina, Noelia

### Mentora:

- Dal Lago, Virginia

### Práctico: Análisis exploratorio y curación

## Motivación

En la actualidad, cada vez más productos se comercializan a través de una plataforma online. Una de las principales ventajas de este sistema es que el usuario puede recibir el producto en su domicilio en una fecha determinada. Pero, ¿cómo sabemos qué día va a llegar? ¿A partir de qué datos podemos predecir la demora del envío? En este práctico se trabajará con datos de envíos de MercadoLibre, el e-commerce más grande de Latinoamérica, analizando y modelando el problema de logística de envíos para poder responder ¿cuándo llega?

## Descripción del dataset

**Datos:**  El conjunto de datos seleccionado para realizar el práctico corresponde a un muestreo aleatorio no uniforme de 500.000 envíos de MercadoLibre. Estos envíos fueron realizados en Brasil en el período comprendido entre Octubre de 2018 y Abril de 2019 (las fechas originales han sido modificadas y adaptadas a un período de tiempo diferente, conservando el día de la semana y considerando los feriados correspondientes). Mientras que las fechas han sido modificadas, los horarios registrados en el dataset son los originales. Los datos comprenden variables tanto categóricas como numéricas. 

El dataset cuenta con las siguientes columnas:

- **Sender_state:** Estado de Brasil de donde sale el envío.
- **Sender_zipcode:** Código postal (de 5 dígitos) de donde sale el envío.
- **Receiver_state:** Estado de Brasil a donde llega el envío.
- **Receiver_zipcode:** Código postal (de 5 dígitos) a donde llega el envío.
- **Shipment_type:** Método de envío (normal, express, super).
- **Quantity:** Cantidad de productos en un envío.
- **Service:** Servicio del correo con el cual se realizó un envío.
- **Status:** Estado del envío (set: listo para ser enviado, sent: enviado, done: entregado, failed: no entregado, cancelled: cancelado).
- **Date_created:** Fecha de creación del envío.
- **Date_sent:** Fecha y hora en que se realizó el envío (salió del correo).
- **Date_visit:** Fecha y hora en que se entregó el envío al destinatario.
- **Shipment_days:** Días hábiles entre que el envío fue enviado (salió del correo) y que fue entregado.

## Objetivos generales


  * Realizar un estudio exploratorio del dataset para extraer información útil sobre el problema a resolver
  * Desarrollar visión crítica en relación a la problemática para llevar a cabo el procedimiento de ciencia de datos
  * Desarrollar habilidades de comunicación de la información obtenida a partir de los datos de manera clara y sencilla.


## Objetivos específicos

ToDo habría que agregar objetivos especificos una vez que tengamos mas avanazado el practico como sugirio la vir en las corecciones

## Metodología

A lo largo de este trabajo trataremos de responder a las siguientes preguntas:

1. ¿Existen datos duplicados? ¿Con qué información deberíamos contar para poder determinar con certeza que dichos datos están duplicados y no corresponden a dos envíos diferentes?
1. En relación a las diferentes columnas de fechas, en el práctico anterior notamos que muchas de ellas no eran consistentes. Identificar y cuantificar los valores atípicos de cada columna. ¿Qué tipo de anomalías hay en el conjunto completo? ¿Qué deberíamos realizar con dichos datos?
1. Una información que, a futuro, podría ser relevante para modelar es el estado de Brasil al cual llega el envío. Sin embargo, esta información está codificada en la columna en formato de texto. ¿De qué maneras podríamos transformar la misma en valores numéricos para utilizar en un modelo? Mostrar al menos dos ejemplos.
1. A la hora de determinar la promesa de entrega de un envío (fecha estimada de llegada), ¿cuáles son los features que consideran pueden tener mayor relevancia? ¿Cuál es el valor a predecir?
1. Suponiendo que queremos emplear un modelo kNN para calcular la promesa de entrega de un envío. ¿Qué transformaciones sugieren realizar sobre los features antes seleccionados? Mostrar al menos un ejemplo.
1. Nos interesa determinar si un envío llegará entre 0-1 días, 2-3 días, 4-5 días, 6-7 días, 8-9 días ó 10 o más días. Construir una nueva columna en el dataset que indique a cuál de estos grupos pertence cada envío, y codificarla numéricamente, de manera tal de poder ser utilizada como valor a predecir. Seleccionar los dos features que se consideren de mayor relevancia para un modelo, y aplicar un procedimiento de clustering (separando en los grupos antes definidos).

Esta comunicación debe estar dirigida para un público técnico pero que desconoce los aspectos propios del problema a resolver (por ejemplo, sus compañeros de clase). Se evaluará, principalmente, la claridad del mensaje presentado, el uso de las herramientas y los conceptos desarrollados en las clases teóricas. 
Además se debe realizar una breve comunicación en pdf (2 páginas máximo) dirigida a un stakeholder del proyecto (por ejemplo, manager), comentando los hallazgos y problemas encontrados, y las posibles acciones a tomar.

## Desarrollo

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy import special
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce # usar "pip install category_encoders" para instalar

BLUE = '#35A7FF'
RED = '#FF5964'
GREEN = '#6BF178'
YELLOW = '#FFE74C'

RELATIVE_PATH = '../diplodatos_2019/mentoria_envios/'
DATA_FILE = 'dataset_sample_corrected.csv'

# Establecemos una semilla por cuestiones de reproducibilidad
np.random.seed(0)

### Lectura y análisis inicial de los datos

In [ ]:
ds = pd.read_csv(RELATIVE_PATH + DATA_FILE, 
                       dtype={'sender_zipcode':'int64',
                              'receiver_zipcode':'int64',
                              'quantity':'int64',
                              'service':'int64'},
                       parse_dates=['date_created','date_sent','date_visit'])

In [ ]:
ds.columns

In [ ]:
ds.info()

In [ ]:
np.random.seed(0) #con propositos de reproducibilidad
ds.sample(10)

###  Shipment_days anómalos

En el práctico anterior habiamos tomado la decisión de no trabajar con valores de la variable **shipment_days** negativos teniendo en cuenta que no los podemos considerar valores reales y que la proporción de estos datos dentro del conjunto total es muy chica comparada con la cantidad total.

Por lo tanto nuestra primer medida será aplicar este filtro.

In [ ]:
m_shipment_days_negative = ds.shipment_days < 0
m_shipment_days_positive = ~ m_shipment_days_negative

ds[m_shipment_days_negative].describe()

In [ ]:
np.random.seed(0)
ds[m_shipment_days_positive].sample(7)

In [ ]:
dataset = ds[m_shipment_days_positive]
dataset.info()

### Datos duplicados

A partir de la información que tenemos del dataset (ver "Descripción del dataset") no se puede identificar una columna que pueda ser tomada como valor único para una fila del dataset. La única columna que quizá pueda tomar esa función, sería **date_created**. Pero, como vimos en el práctico anterior, esta variable sólo tiene fecha (y no hora) de la creación del evento, con lo cual no tiene sentido tomarla como "id" en nuestro dataset.

Con lo cual, una siguiente hipótesis sería tomar a todos los datos del dataset como el identificador de un envío en particular. Veamos:

In [ ]:
ds[ds.duplicated(keep=False)].sort_values('date_created', ascending=True)

Tenemos entonces 11863 filas del dataset (2.37%) que son iguales, dato a dato, a alguna otra fila. La pregunta que corresponde hacerse es: ¿Es suficiente saber que dos filas son iguales dato a dato para considerarlas iguales, o podríamos considerar por error que son datos iguales cuando corresponden a envíos distintos?

A priori, creemos que no hay una respuesta que podamos dar que tenga una certidumbre razonable para tomar una decisión, sin contar con el aporte de información de expertos en el dominio (esto es, personas que sepan cómo estos datos fueron tomados o generados, qué tipo de error tiene cada uno de los datos, cuáles datos son generados automáticamente y cuáles manualmente, etc.) o información en el dataset que sean indicadores determinado para esta unicidad (como por ejemplo, un identificador -una o más variables del dataset- declarado como tal).

Intentaremos sumar un poco más de información a partir de la exploración del dataset y de los (potencialmente) duplicados para tomar una decisión informada:

1. Lo primero que vemos es que tanto **date_sent** como **date_received** tienen precisión al segundo. Podríamos asumir que aquellas entradas con el valor del segundo en 0 para algunos de estos campos podrían responder a que la forma de carga toma información hasta el minuto, pero vemos ocurrencias de entradas con segundos distintos de 0, con lo cual esta asunción no tendría mucho sustento (o al menos no lo tendría para muchos de los casos).
Con lo cual, la probabilidad de que dos paquetes que salen del mismo estado, van al mismo estado, usan el mismo correo, y llegan y son recibidos a una misma fecha (con precisión de segundos, más allá de si este dato en particular se refiere al momento de carga o al momento preciso de envío o recepción) consideramos que es realmente baja. Este punto apoyaría la moción de considerar como repetidas estas entradas.

2. Para el caso de envíos con **quantity** mayor a 1, esta moción se reforzaría, ya que los envíos con más de un paquete son una parte minoritaria de los envíos, y la probabilidad de que no fuera un duplicado sería aún más baja.

3. Mismo análisis que el punto anterior se puede hacer para el caso de **shipment_type** super express.

A partir de este análisis, creemos que tiene sentido considerar a las filas que comparten el mismo valor para todas sus columnas como repetidas y no como envíos distintos. O, en todo caso, considerarlos como datos que no son confiables para la problemática que estamos intentando atacar.

Siendo que estas filas pueden estar repetidas más de dos veces, intentemos cuantificar de cuántas estamos hablando:

In [ ]:
ds[ds.duplicated()].sort_values('date_created', ascending=True)

Listando sólo una ocurrencia de las filas repetidas (en este caso la 1era, de acuerdo a cómo funciona *duplicated*) tenemos 6483 filas únicas del original 11863 que incluyen todas las repetidas. Esto nos da un total de 5380 filas que estaríamos considerando repetidas y debemos quitar del dataset como parte de nuestro proceso de curación.

### Datos faltantes

Lo siguiente que vamos a revisar, como lo hicimos en el práctico anterior, son los datos faltantes (para cualquiera de las columnas del dataset). Intentaremos listarnos, buscar una explicación de los mismos, y decidir si (y cómo) lidiamos con ellos:

In [ ]:
ds.isnull().sum()

Pandas reconoce 1233 valores faltantes, en las variables **date_sent** (29), **date_visit** (602) y **shipment_days** (602). Esperaríamos a priori que:
1. Las filas con **shipment_days** faltantes y las filas con **date_visit** faltantes sean las mismas;
1. Que los valores faltantes se encuentren en envíos con un **status** que explique este faltante:

In [ ]:
filter_1 = ds.shipment_days.isnull() | ds.date_visit.isnull()
len(ds[filter_1])

Validamos el primer supuesto. Veamos el segundo:

In [ ]:
filter_2 = ds.date_sent.isnull() | ds.date_visit.isnull()
ds[filter_2].status.unique()

No validamos el segundo supuesto, llama la atención los envíos en "done" con estas condiciones, con lo que podemos concluir que pueden ser anómalos:

In [ ]:
ds[filter_2][ds.status == 'done']

Intentaremos ver si existen otros valores "faltantes" que Pandas no esté reconociendo como tales:

In [ ]:
# ToDo: Esto se puede hacer un poco más prolijo
for col in ds:
    print (ds[col].unique())

A simple vista, pareciera que todos los valores faltantes están considerados en el análisis anterior.

Dicho esto, y con el contexto de la problemática que deseamos resolver, creemos que es conveniente eliminar las filas con datos faltantes: responden a datos que no nos servirán de información para la estimación de envío (son envíos que no llegaron, por diversas razones) o son datos que consideramos anómalos para el dataset. Cualquier política para completar esos datos con algún valor distinto (persistiendo la fila en cuestión) creemos que nos llevaría a tener información no confiable en el dataset.

### Datos anómalos

Más allá de los valores anómalos mencionados en la sección anterior, intentaremos identificar otras anomalías o inconsistencias en el data set.

**ToDo: Acá copiaría exactamente lo que hicimos el práctico anterior ("Análisis de shipment_days anómalos" y "Análisis de fechas anómalas")**


### Análisis de fechas anómalas (Parte I)


Las 3 variables de fechas que tenemos en este dataset, y los supuestos que proponemos para ellas, son:

- **date_created**: Corresponde a la fecha de creación del envío.
  - Es una fecha que se crea automáticamente a través de algunos de los procesos de venta de MercadoLibre.
  - Sólo nos interesa la fecha y no la hora de creación
- **date_sent**: Corresponde a la fecha y hora en que alguno de los correos cargó para el envío en cuestión, que comenzó el proceso de envío
 - Consideraremos **date_sent** como la fecha en que el envío está a cargo del correo y no necesariamente cuando el envío "salió" del correo (más bien cuando "entró" al correo, que es cuando el proceso de handling llega a su fin).
 - Puede crearse "automáticamente" cuando el vendedor deja el contenido del envío en el correo.
  - Puede también crearse "manualmente", pensando en lugares de recepción de correos sin la tecnología suficiente para que esta fecha sea generada automáticamente. Es posible entonces que sea necesario "aceptar" algunas fechas no precisas como válidas.
 * Por los supuestos en **date_created** y las anteriores para **date_sent**, esta fecha debe necesariamente ser posterior a **date_created**
- **date_visit**: Corresponde a la fecha y hora en que alguno de los correos cargó en envío en cuestión como entregado.
  - Puede crearse "automáticamente" cuando el cartero utiliza tecnología (lector de código de barras o aplicación movil, por ejemplo).
  - Puede crearse "manualmente", pensando en que el cartero, al terminar un ciclo o jornada, provee de la información de todos los envíos entregados en ese ciclo o jornada.
  - Por los supuestos en **date_created**, **date_sent** y los anteriores de **date_visit**, esta fecha debe neceariamente ser posterior a **date_created**. Se espera también que sea posterior a **date_sent** también, pero deberemos tomar una decisión en estos casos en función de análisis del dataset.

In [ ]:
m_date_created_after_date_sent = (ds.date_created >= ds.date_sent)
m_date_created_before_date_sent = (ds.date_created < ds.date_sent)

m_date_created_after_date_visit = (ds.date_created >= ds.date_visit)
m_date_created_before_date_visit = (ds.date_created < ds.date_visit)

m_date_created_invalid = m_date_created_after_date_sent | \
                         m_date_created_after_date_visit

ds[m_date_created_invalid].describe()

Tenemos 194 casos donde los datos no siguen los supuestos planteados. Por lo tanto, consideramos que esta información anómala no es confiable y deberíamos eliminarla del dataset.

In [ ]:
m_date_sent_after_date_visit = ds.date_sent >= ds.date_visit
m_date_sent_before_date_visit = ds.date_sent < ds.date_visit

ds[m_date_sent_after_date_visit].describe()

Tenemos 2407 casos en donde se consignó que el paquete entró al proceso de envío después de la fecha de consignación de llegada. No representan una cantidad significativa de casos como para analizar si tiene sentido sacarlos o dejarlos del dataset, por lo que consideramos que es información no confiable que podemos quitar.

### Análisis de fechas anómalas (Parte II)

In [ ]:
dataset.describe(include='datetime64')

En el práctico anterior habiamos observado lo siguiente:

* La variables **date_created** solo tiene 144 valores distintos y no considera horas (todas están seteadas a las 00:00 hs) esto implica que estamos considerando un total de 144 días distintos.
* La variable **date_created**  contienen fechas que caen fuera del intervalo temporal considerado y más aún, que representa una fecha futura (Junio de 2019).
* Hay una cierta variabilida entre los mínimos de las 3 fechas, podría servir analizar los estados de las fechas fuera del periodo de **date_visit** para ver si no son anómalos.
* En la variable **date_visit** el dato más frecuente se da a las 22.00 hs, posiblemente fuera del horario de atención de los correos, con lo cual este dato es bastante extraño en un principio. Lo mismo sucede con el horario 05:36.

Teniendo en cuenta la problemática que queremos resolver, la primera observación cobra cierto sentido pues el día de creación del envío resulta mucho más importante que la hora en si misma. Además, considerando que la respuesta del problema en un principio debe consistir en dar un intervalo de días, las franjas horarias podrían no ser tenidas en cuenta a priori.

Con respecto a la segunda observación podríamos intentar determinar la cantidad de fechas que caen fuera del intervalo Octubre de 2018-Abril de 2019 y al mismo tiempo observar la tendencia a medida que pasan los meses de las variables **date_created**, **date_sent** y **date_visited**.



In [ ]:
dataset['date_created'].groupby(
    [dataset['date_created'].dt.year.rename('year'),
     dataset['date_created'].dt.month.rename('month')]).agg({'count'})


**Observaciones:**

* Tenemos una fecha de creación en el mes de septiembre y 111 fechas en el mes de Junio . Todas ellas caen fuera del intervalo considerado.

* La mayor concentración en cuanto a fechas de creación de envíos se dió en la franja enero-marzo, donde marzo cuenta con más de la mitad de los datos. 

**Preguntas y comentarios a tener en cuenta:**

1. ¿Es normal que en un intervalo de 6 meses los datos se concentren mayormente en un mes o dos?
2. ¿Podemos pensar que los datos de septiembre de 2018 y julio de 2019 hayan tenido que ver con errores de carga de los datos y sean efectivamente datos que se encuentran en el intervalo considerado? ¿O efectivamente serán datos que tenemos que eliminar de nuestro conjunto de análisis?
3. Los 111 datos en el mes de junio resultan bastante extraños y quizás merecen un análisis más detallado.

In [ ]:
dataset['date_sent'].groupby(
    [dataset['date_sent'].dt.year.rename('year'),
     dataset['date_sent'].dt.month.rename('month')]).agg({'count'})


**Observaciones:**

No se observan datos fuera del intervalo considerando pero si  una gran concentración en el intervalo enero-marzo.



In [ ]:
dataset['date_visit'].groupby(
    [dataset['date_visit'].dt.year.rename('year'),
     dataset['date_visit'].dt.month.rename('month')]).agg({'count'})


**Observaciones:**

* No se observan datos anómalos en la variable **date_visit**.
* Resulta extraño que tengamos fecha de visita solo en los meses de febrero, marzo y abril cuando nuestro intervalo temporal es más grande
* Estos datos merecen un análisis más detallado.



### Análisis de la variable date_created

Ahora vamos a analizar las 111 fechas de creación del mes de junio y el valor del mes de septiembre para ver si encontramos alguna anomalia.

In [ ]:
date_created_septiembre = dataset['date_created'].map(lambda x: x.month) == 9
dataset[date_created_septiembre]

Consideramos que este dato está mal cargado y su presencia no aporta valor a nuestro análisis. Por lo tanto procederemos a eliminarlo.

In [ ]:
dataset.drop(dataset[dataset['date_created'].map(lambda x: x.month) == 9].index, inplace=True)

In [ ]:
dataset[date_created_septiembre] #Corroboramos que se haya eliminado

In [ ]:
date_created_junio = dataset['date_created'].map(lambda x: x.month) == 6
dataset[date_created_junio]

Si bien estos datos caen fuera de los supuestos en cuanto a que la fecha de creación debe ser anterior a la fecha de envío, la inspección de los datos de arriba  nos inclina a pensar que los datos con fecha de creación en junio de 2019 en realidad corresponden a datos del mes de enero del mismo año. De esa manera, las columnas de las fechas parecerian tener más coherencia y cumplir con los supuestos planteados. Bajo esta hipótesis nos encontramos frente a la diyuntiva de si eliminamos esta información o imputamos los datos haciendo que su fecha de creación sea  enero de 2019.

Si uno se guía por los datos como vienen la primera opción sería estas filas (en cuanto a cantidad no afectaría a la cantidad total) sin embargo podemos tomar la jugada arriesgada y "modificar" nuestros datos subsanando de esta manera en posible error producido durante la carga de los datos. 

A continuación reemplazaremos los datos de creación de junio de 2019 por enero de 2019.

## Dataset limpio

#ToDo 
(habría que charlar esta parte en base a la parte II de arriba)

En función de lo visto en las secciones anteriores, generaremos un nuevo dataset con las siguientes características:

- Sin datos "potencialmente duplicados" (ver "Datos duplicados")
- Sin datos nulos (ver "Datos faltantes")
- Sin **shipment_days** negativos (ver "Análisis de shipment_days anómalos")
- Sin envíos creados después de ser enviados (ver "Análisis de fechas anómalas")
- Sin envíos creados después de ser recibidos (ver "Análisis de fechas anómalas")
- Sin envíos enviados después de ser recibidos (ver "Análisis de fechas anómalas")

In [ ]:
# Generamos un nuevo dataset "limpio"

# 1. Sin datos "potencialmente duplicados"
dataset = ds.drop_duplicates()

# 2. Sin datos nulos
dataset = dataset.dropna()

# 3. Sin shipment_days negativos
dataset = dataset[m_shipment_days_positive]

# 4. Sin envíos creados después de ser enviados
dataset = dataset[m_date_created_before_date_sent]

# 5. Sin envíos creados después de ser recibidos
dataset = dataset[m_date_created_before_date_visit]

# 6. Sin envíos enviados después de ser recibidos
dataset = dataset[m_date_sent_before_date_visit]

dataset.to_csv(RELATIVE_PATH + "data_sample_corrected_cleaned.csv", sep=',', index=False)
dataset.describe()

Resultan 490.439 entradas (de las 500.000 originales), con lo cual hemos dejado en el camino menos del 2% del dataset original, un número que, por la confianza que nos transmiten esos datos, podemos anticipar que es razonable.

## Pregunta 3

Para convertir datos categóricos en numéricos se pueden usar, entre otros, los siguientes dos métodos:
* **One-Hot encoding:** traduce cada valor categórico en una columna independiente con valor 1 en los registros que tenían dicha categoría. Este método, si bien no tiene problemas con los "pesos" que pueden heredar las categorías, puede presentar problemas cuando el número total de categorías es grande, ya que los datos se codifican en grandes dimensiones.
* **Binary Encoding:** toma el índice ordinal de las categorías y las codifica de forma binaria, donde los bits de esa codificación se encuentran separados en columnas. La ventaja que tiene respecto al método anterior es que los datos se codifican en menores dimensiones.

In [ ]:
dataset['receiver_state'] = pd.Categorical(dataset['receiver_state'])


In [ ]:
dataset.info()

**One-Hot encoding**

In [ ]:
dfDummies = pd.get_dummies(dataset['receiver_state'], prefix = 'category')


In [ ]:
dataset_dummies = pd.concat([dataset,dfDummies], axis=1)
dataset_dummies.sample(10)

**Binary Encoding**

In [ ]:
dataset_ce = dataset.copy()

encoder = ce.BinaryEncoder(cols=['receiver_state'])
dataset_binary = encoder.fit_transform(dataset_ce)

dataset_binary.head()